<img src="img/mmselfsup_logo.png">

# 图片向量可视化 t-SNE

<a href="https://colab.research.google.com/github/wangruohui/OpenMMLabCourse/blob/main/codes/MMSelfSup_tutorials/【2】图片向量可视化%20t-SNE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**MMSelfSup Repo**：[https://github.com/open-mmlab/mmselfsup](https://github.com/open-mmlab/mmselfsup)

**MMSelfSup 官方文档链接**：[https://mmselfsup.readthedocs.io/en/latest](https://mmselfsup.readthedocs.io/en/latest)

**作者**：OpenMMLab

## 0. 任务介绍

**t-SNE** 是一种数据降维与可视化的方法。当我们想对高维数据进行分类时，可以先对其使用 t-SNE 进行可视化，看其是否具备可分性。具体来说，就是将数据通过 t-SNE 投影到二维或者三维空间中，如果在低维空间中同类数据间隔小，异类之间间隔大，那么说明数据是可分的；但如果在低维空间中不具有可分性，可能是数据本身不可分，也有可能是因为其不能投影到低维空间中。

我们经常使用 t-SNE 可视化来展示自监督预训练的学习效果。如果模型能在自监督预训练过程中学习到比较好的特征提取能力，那么数据经过该模型提取到的高维特征就应该具有可分性。

本教程将演示：使用 t-SNE 可视化模型自监督预训练的学习效果

代码详细请参考官方[文档](https://mmselfsup.readthedocs.io/zh_CN/latest/get_started.html#t-sne)。

## 1. 环境配置

### 1.1 查看 Python、PyTorch 和 Torchvision 的版本

In [ ]:
# Check nvcc version
!nvcc -V

In [ ]:
# Check GCC version
!gcc --version

In [ ]:
# Check PyTorch installation
import torch, torchvision
print(torch.__version__)
print(torch.cuda.is_available())

### 1.2 安装 MMSelfSup 的依赖库：MMCV

In [ ]:
!pip install openmim

In [ ]:
!mim install mmcv

### 1.3  安装 MMSelfSup

In [ ]:
%cd /content

In [ ]:
!git clone https://github.com/open-mmlab/mmselfsup.git
%cd /content/mmselfsup

In [ ]:
# Install MMSelfSup from source
!pip install -e . 

### 1.4 检查安装是否正确

In [ ]:
import mmselfsup
print(mmselfsup.__version__)

## 2. 准备数据集

### 2.0 数据集介绍

本教程将使用自监督预训练模型提取 `Tiny ImageNet` 数据集里的图片特征，并进行 t-SNE 可视化。

Tiny ImageNet 数据集是 ImageNet 的一个子集。

该数据集包含 200 个类别，每个类别有 500 张训练图片、50 张验证图片和 50 张测试图片，共 120,000 张图像。每张图片均为 64×64 彩色图片。

数据集官方下载地址：http://cs231n.stanford.edu/tiny-imagenet-200.zip

### 2.1 下载数据集

使用 GNU [Wget](https://www.gnu.org/software/wget/) 工具从斯坦福官方网站下载：http://cs231n.stanford.edu/tiny-imagenet-200.zip

In [ ]:
%cd /content/mmselfsup

In [ ]:
!mkdir data
%cd data
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip

### 2.2 解压数据集

In [ ]:
!unzip -q tiny-imagenet-200.zip

In [ ]:
!rm -rf tiny-imagenet-200.zip

### 2.3 查看数据集目录

In [ ]:
# Check data directory
!apt-get install tree
!tree -d /content/mmselfsup/data

### 2.4 准备标注文件

为了减少大家重写 `加载数据集` 代码的负担，我们整理好了标注文件，复制到数据集根目录 `mmselfsup/data/tiny-imagenet-200` 下即可。

In [ ]:
%cd /content/mmselfsup/data

In [ ]:
!wget https://raw.githubusercontent.com/wangruohui/OpenMMLabCourse/main/codes/MMSelfSup_tutorials/anno_files/train.txt -P tiny-imagenet-200
!wget https://raw.githubusercontent.com/wangruohui/OpenMMLabCourse/main/codes/MMSelfSup_tutorials/anno_files/val.txt -P tiny-imagenet-200

## 3. 准备自监督预训练模型

目前，MMSelfSup 里 t-SNE 可视化的模型对象既可以是 `自监督预训练模型提取 backbone 部分的权重文件`，也可以是 `自监督预训练过程中直接保存下来的名为 epoch_*.pth 的 ckeckpoint 文件`。所以，我们这里需要准备好两种模型文件，方便后面对这两种文件进行 t-SNE 可视化的代码演示。

**注意**：目前，MMSelfSup 只支持以 `ResNet-50` 为 backbone 的自监督预训练模型的 t-SNE 可视化。

### 3.1 准备自监督预训练模型提取 backbone 部分的权重文件

**注意：MMSelfSup 的 [模型库](https://github.com/open-mmlab/mmselfsup/blob/master/docs/en/model_zoo.md) 中的模型文件都已经提取过 backbone 权值，不需要再次提取！**我们直接使用即可。

在模型库中找到在 SimCLR 的预训练模型文件 `simclr_resnet50_8xb32-coslr-200e_in1k`，下载放在 `checkpoints` 文件夹里 

In [ ]:
%cd /content/mmselfsup
!mkdir checkpoints
!wget https://download.openmmlab.com/mmselfsup/simclr/simclr_resnet50_8xb32-coslr-200e_in1k_20220428-46ef6bb9.pth -P checkpoints

### 3.2 准备自监督预训练过程中直接保存下来的 ckeckpoint 文件

我们使用第一个教程 `模型自监督预训练 之 SimCLR` 中训练保存下来的 `epoch_1.pth` 文件进行演示，该文件可以从 [这里](https://download.openmmlab.com/mmselfsup/tutorial/epoch_1.pth) 下载，存放在文件夹 `mmselfsup/work_dirs/selfsup/simclr/simclr_resnet50_1xb32-coslr-1e_tinyin200` 里。

In [ ]:
%cd /content/mmselfsup
!mkdir -p work_dirs/selfsup/simclr/simclr_resnet50_1xb32-coslr-1e_tinyin200
!wget https://download.openmmlab.com/mmselfsup/tutorial/epoch_1.pth  -P  work_dirs/selfsup/simclr/simclr_resnet50_1xb32-coslr-1e_tinyin200

同时，准备好第一个教程 `模型自监督预训练 之 SimCLR` 中的自监督预训练配置文件 `simclr_resnet50_1xb32-coslr-1e_tinyin200.py`。

In [ ]:
%cd /content/mmselfsup

In [ ]:
%%writefile /content/mmselfsup/configs/selfsup/simclr/simclr_resnet50_1xb32-coslr-1e_tinyin200.py

_base_ = 'simclr_resnet50_8xb32-coslr-200e_in1k.py'

# dataset
data = dict(
    samples_per_gpu=32, 
    workers_per_gpu=2,
    train=dict(
        data_source=dict(
            data_prefix='data/tiny-imagenet-200/train',
            ann_file='data/tiny-imagenet-200/train.txt',
        )
    )
)

# optimizer
optimizer = dict(
    lr=0.3 * ((32 * 1) / (32 * 8)),
)

runner = dict(max_epochs=1)

## 4. 写 t-SNE 可视化的数据配置文件

1. 新建一个名为 `tsne_tinyin200` 的配置文件。



2. `tsne_tinyin200` 数据配置文件的内容如下：
    1. 继承 [tsne_imagenet.py](https://github.com/open-mmlab/mmselfsup/blob/master/configs/benchmarks/classification/tsne_imagenet.py) 配置文件
    2. 修改数据集路径和数据标注文件路径

In [ ]:
%cd /content/mmselfsup

In [ ]:
%%writefile /content/mmselfsup/configs/benchmarks/classification/tsne_tinyin200.py
_base_ = 'tsne_imagenet.py'

data = dict(
    extract=dict(
        data_source=dict(
            data_prefix='data/tiny-imagenet-200/val',
            ann_file='data/tiny-imagenet-200/val.txt',
        )
    )
)

## 5. 使用 t-SNE 可视化自监督预训练模型提取的图片特征

下面，我们分别演示对 `自监督预训练模型提取 backbone 部分的权重文件` 和 `自监督预训练过程中直接保存下来的名为 epoch_*.pth 的 ckeckpoint 文件` 两种文件进行 t-SNE 可视化。

### 5.1 对自监督预训练模型的 backbone 权重文件提取图片特征并保存 t-SNE 可视化结果

使用 [visualize_tsne.py](https://github.com/open-mmlab/mmselfsup/blob/master/tools/analysis_tools/visualize_tsne.py) 脚本来提取图片特征并保存 t-SNE 可视化结果。

```shell
python tools/analysis_tools/visualize_tsne.py ${CONFIG_FILE} --dataset_config ${DATASET_CONFIG} --cfg-options ${CFG_OPTION} --work-dir ${WORK_DIR} [optional arguments]
```

参数:
+ CONFIG_FILE：自监督训练的配置文件所在路径
+ dataset_config：数据配置文件所在路径
+ cfg-options：配置文件的可选项。如果要对“自监督预训练模型的 backbone 权重文件”进行 t-SNE 可视化，就在该参数里进行设置。
+ work-dir：保存可视化结果的路径

其他可选参数 optional arguments 可参考 [visualize_tsne.py](https://github.com/open-mmlab/mmselfsup/blob/master/tools/analysis_tools/visualize_tsne.py).

In [ ]:
%cd /content/mmselfsup

In [ ]:
!python tools/analysis_tools/visualize_tsne.py \
configs/selfsup/simclr/simclr_resnet50_8xb32-coslr-200e_in1k.py \
--dataset_config configs/benchmarks/classification/tsne_tinyin200.py \
--cfg-options model.backbone.init_cfg.type=Pretrained \
model.backbone.init_cfg.checkpoint=checkpoints/simclr_resnet50_8xb32-coslr-200e_in1k_20220428-46ef6bb9.pth \
--work-dir work_dirs/selfsup/simclr_resnet50_8xb32-coslr-200e_in1k/tsne_simclr

### 5.2 对自监督预训练过程中直接保存下来的 ckeckpoint 文件提取图片特征并保存 t-SNE 可视化结果

使用 [visualize_tsne.py](https://github.com/open-mmlab/mmselfsup/blob/master/tools/analysis_tools/visualize_tsne.py) 脚本来提取图片特征并保存 t-SNE 可视化结果。

```shell
python tools/analysis_tools/visualize_tsne.py ${CONFIG_FILE} --dataset_config ${DATASET_CONFIG} --checkpoint ${CKPT_PATH} --work-dir ${WORK_DIR} [optional arguments]
```

参数:
+ CONFIG_FILE：自监督训练的配置文件所在路径
+ dataset_config：数据配置文件所在路径
+ checkpoint：自监督预训练过程中保存下来（名为 `epoch_*.pth`）的模型文件路径
+ work-dir：保存可视化结果的路径

其他可选参数 optional arguments 可参考 [visualize_tsne.py](https://github.com/open-mmlab/mmselfsup/blob/master/tools/analysis_tools/visualize_tsne.py).

In [ ]:
%cd /content/mmselfsup

In [ ]:
!python tools/analysis_tools/visualize_tsne.py \
configs/selfsup/simclr/simclr_resnet50_8xb32-coslr-200e_in1k.py \
--dataset_config configs/benchmarks/classification/tsne_tinyin200.py \
--checkpoint work_dirs/selfsup/simclr/simclr_resnet50_1xb32-coslr-1e_tinyin200/epoch_1.pth \
--work-dir work_dirs/selfsup/simclr_resnet50_8xb32-coslr-200e_in1k/tsne_simclr

### 5.3 对权值随机初始化的模型提取图片特征并保存 t-SNE 可视化结果

同时，我们可以和 `同一个 backbone 但是权值随机初始化` 的模型进行对比，对比二者提取图片特征的 t-SNE 可视化效果。

做法：既不设置 checkpoint 参数，也不设置 cfg-options 参数。

In [ ]:
!python tools/analysis_tools/visualize_tsne.py \
configs/selfsup/simclr/simclr_resnet50_8xb32-coslr-200e_in1k.py \
--dataset_config configs/benchmarks/classification/tsne_tinyin200.py \
--work-dir work_dirs/selfsup/simclr_resnet50_8xb32-coslr-200e_in1k/tsne_random

## 6. 显示 t-SNE 可视化图片

根据上面运行的结果信息，修改 t-SNE 可视化图片的目录路径

In [ ]:
selfsup_tsne_dir = 'work_dirs/selfsup/simclr_resnet50_8xb32-coslr-200e_in1k/tsne_simclr/tsne_20220719_131352/saved_pictures/'
rand_tsne_dir = 'work_dirs/selfsup/simclr_resnet50_8xb32-coslr-200e_in1k/tsne_random/tsne_20220719_131509/saved_pictures/'

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 50), constrained_layout=True)
for i in range(1, 6):
    # the full path of images
    selfsup_tsne_feat_i_path  = selfsup_tsne_dir + 'feat'+ str(i) + '.png'
    rand_tsne_feat_i_path  = rand_tsne_dir + 'feat'+ str(i) + '.png'
    
    # open the images
    selfsup_tsne_feat_i_images = Image.open(selfsup_tsne_feat_i_path)
    rand_tsne_feat_i_images = Image.open(rand_tsne_feat_i_path)
    
    # plot the images
    plt.subplot(5, 2, 2*i-1)
    plt.title('selfsup feat '+ str(i) +' tsne', y=0.9, fontsize=30)
    plt.imshow(selfsup_tsne_feat_i_images)
    plt.axis('off')
    
    plt.subplot(5, 2, 2*i)
    plt.title('random feat '+ str(i) +' tsne', y=0.9,  fontsize=30)
    plt.imshow(rand_tsne_feat_i_images)
    plt.axis('off')

plt.show()